In [5]:
import requests
import os
import datetime
import json

In [3]:
def get_access_token():
    client_id = os.environ["BASECAMP_CLIENT_ID"]
    client_secret = os.environ["BASECAMP_CLIENT_SECRET"]
    redirect_url = os.environ["BASECAMP_REDIRECT_URL"]
    refresh_token = os.environ["BASECAMP_REFRESH_TOKEN"]
    
    refresh_token_url = f"https://launchpad.37signals.com/authorization/token?type=refresh&refresh_token={refresh_token}&client_id={client_id}&redirect_uri={redirect_url}&client_secret={client_secret}"
    access_token = requests.post(refresh_token_url ).json()['access_token']
    return access_token

In [6]:
ACCOUNT_ID = "3694081"
BASE_URL = f"https://3.basecampapi.com/{account_id}"
project_id = "######" # Enter project id
message_board_id = "######" # Enter message board id

In [7]:
def get_sgid(path):
    attachments_url = f"{base_url}/attachments.json?name={path}"
    file_size = os.path.getsize(path)
    img_headers = {
        'Authorization': 'Bearer '+ get_access_token(),
        "Content-Type": "image/png",
        "Content-Length": str(file_size)
        }
    
    with open(path, "rb") as img_content:
        image_sgid = requests.post(attachments_url, headers=img_headers, data=img_content).json()['attachable_sgid']
        
    return image_sgid

In [8]:
class metrics:
    date = datetime.date.today() - datetime.timedelta(1)
    day_of_month = date.day
    year_month = date.strftime('%B %Y')
    bc_post_name = f"Daily Metrics - {year_month}"

In [11]:
def create_bc_post():
    url = f"{base_url}/buckets/{project_id}/message_boards/{message_board_id}/messages.json"
    headers = {
        "Authorization": "Bearer " + get_access_token(),
        "Content-Type": "application/json"
    }
    
    content = "<p><strong>Daily metrics for "+metrics.year_month+" will be posted in the comment section.</strong></p><br> \
    <p>Explore metrics in the <a href='https://ed8a34f1.us2a.app.preset.io/superset/dashboard/p/GRypvmnXzMl/'>KPI Dashboard</a>.<p> \
    <p>Learn about metrics on the <a href='https://docs.fishingbooker.org/data/daily-metrics'>documentation page</a>.</p><br> \
    <p>🚀 <em><span style='color: rgb(17, 138, 15);'>Boosts</span> are encouraged!</em></p> \
    <p>💬 <em><span style='color: rgb(207, 0, 0);'>Comment</span> only if necessary to avoid spamming everyone in the company.</em></p><br> \
    <p><em>If you notice something wrong with the metrics, you can ping me or write in Data team's <a href='https://3.basecamp.com/3694081/buckets/22348834/todolists/5214969729'>Inbox</a> or <a href='https://3.basecamp.com/3694081/buckets/22348834/chats/3779965909'>Campfire</a>.</em></p>"
    
    data = str('{"subject": "'+metrics.bc_post_name+'", "content": "'+content+'", "status": "active"}').encode()

    requests.post(url, headers=headers, data=data)

In [10]:
def post_metrics():
    message_board_url = f"{base_url}/buckets/{project_id}/message_boards/{message_board_id}/messages.json"
    headers = {
        "Authorization": "Bearer " + get_access_token(),
        "Content-Type": "application/json"
    }
    messages = requests.get(message_board_url, headers=headers).json()
    
    if not(any(msg['subject'] == metrics.bc_post_name for msg in messages)):
        create_bc_post()
        messages = requests.get(message_board_url, headers=headers).json()
        
    for msg in messages:
        if msg['subject'] == metrics.bc_post_name:
            message_id = msg['id']
    
    comment_url = f"{base_url}/buckets/{project_id}/recordings/{message_id}/comments.json"
    
    images = []
    for file in sorted(os.listdir("/tmp/images")):
        if file.endswith(".png"):
            image = {
                "name": os.path.basename(file),
                "sgid": get_sgid(f"/tmp/images/{file}")
            }
            images.append(image)
    
    content = ''
    for image in images:
        content += "<bc-attachment sgid='"+image['sgid']+"'></bc-attachment> \
            <br> \
            "
    content += "- <a href='https://ed8a34f1.us2a.app.preset.io/superset/dashboard/p/GRypvmnXzMl/'>KPI Dashboard</a> <br> \
        - <a href='https://datastudio.google.com/reporting/8c88333b-a69b-47ae-8dcd-9163a8732b09'>Additional metrics dashboard</a> <br> \
        - <a href='https://docs.fishingbooker.org/data/daily-metrics'>Metrics documentation</a> <br><br> \
        <em style='color: rgb(102, 102, 102);'>This is an automated post. If you notice something wrong with the metrics, you can ping me or write in Data team's <a href='https://3.basecamp.com/3694081/buckets/22348834/todolists/5214969729'>Inbox</a> or <a href='https://3.basecamp.com/3694081/buckets/22348834/chats/3779965909'>Campfire</a>.</em>"
    
    requests.post(comment_url, headers=headers, data='{"content": "'+content+'"}')